In [1]:
import csv
from collections import defaultdict
import math
import pprint as pp

In [24]:
def rfid_info(filename):
    rfids = defaultdict(list)
    with open(filename) as f:
        csv_reader = csv.reader(f, delimiter=',')
        line_count = 0

        rfid_start = 1 << 64
        rfid_end = 0
        for row in csv_reader:
            epc = row[0] # electronic product code
            peakRssiInDbm = float(row[1]) # Received Signal Strength Indicator
            phase_diff1 = float(row[2]) # Phase difference between [WHAT]
            rfDopplerFreq = row[3] # 
            channelMhz = float(row[4]) # Which channel it was on
            time_millis = float(row[5]) / 1000
            rfid_start = min(rfid_start, float(time_millis))
            rfid_end = max(rfid_end, float(time_millis))
            data = {'peak_rssi': peakRssiInDbm, 'phase_diff': phase_diff1, 'rfDopplerFreq': rfDopplerFreq, 'channelMhz': channelMhz, 'time_millis': time_millis}
            rfids[epc].append(data)
    return rfids

def select_useful_rfid(rfid_info):
    return rfid_info['E20038EBB5953DC9B7693E9C']

def rfid_avgs(rfids):
    avg_rfids_rev = {k: sum([x['peak_rssi'] for x in v]) / len(v) for k, v in rfids.items()}
    return avg_rfids_rev

def wavelength(freq_mhz):
    return 3 * (10 ** 2) / freq_mhz
        
def rfid_dists(measurements):
    epc_dists = {}
    dists = []
    dist = 0 # might need to change to initial phase
    for measurement in measurements:
        wave_len = wavelength(measurement['channelMhz'])
        dists.append(dist)
#             import pdb; pdb.set_trace()
#         print('phase_diff', measurement['phase_diff'])
        dist += measurement['phase_diff'] % (2 * math.pi) * wave_len
        epc_dists[epc] = dists
#         print()
    
    return epc_dists

def img_metadata(filename):
    timestamps = [float(time) for time in open(filename, 'r').readlines()]
    return timestamps

# Array is sorted
def idx_find(array, target, start=0):
    if start >= len(array):
        return -1
    for idx, val in enumerate(array):
        if idx >= start:
            if val >= target:
                return idx
    return -1

In [37]:
experiment_name = 'stationary_center'
rfid_data = rfid_info(f'data/{experiment_name}/tag_data.csv')
img_data = img_metadata(f'data/{experiment_name}/{experiment_name}_meta.txt')
disc1_tag = select_useful_rfid(rfid_data)
disc_time_data = [tag_data['time_millis'] for tag_data in disc1_tag]

# Want to decide which image an RFID reading goes to
# So iterate through RFID readings, choose the img that the reading is closest to

tags_start = idx_find(disc_time_data, img_data[0] - (img_data[1] - img_data[0]) / 2)
tags_end = idx_find(disc_time_data, img_data[-1] + (img_data[-1] - img_data[-2]) / 2)

prev_start = 0
img2tag_mapping = defaultdict(list)

img_idx = 0
for tag_idx, tag_time in enumerate(disc_time_data[tags_start:tags_end+1]):
    closest_img, smallest_dist = None, float('inf')
    for d_idx in [-1, 0, 1]:
        if 0 <= img_idx + d_idx < len(img_data):
            if abs(img_data[img_idx + d_idx] - tag_time) < smallest_dist:
                smallest_dist = abs(img_data[d_idx] - tag_time)
                closest_img = img_idx + d_idx
    if closest_img == img_idx + 1:
        img_idx += 1
    img2tag_mapping[closest_img].append(tags_start + tag_idx)


for idx, t_i in enumerate(img_data):
    if idx == len(img_data) - 1:
        gap = img_data[idx] - img_data[idx - 1]
        start_time = t_i - gap / 2
        end_time = t_i + gap / 2
    else:
        gap = img_data[idx + 1] - img_data[idx]
        start_time = t_i - gap / 2
        end_time = t_i + gap / 2
    
    bucket_start = idx_find(disc_time_data, start_time, prev_start)
    bucket_end = idx_find(disc_time_data, end_time, bucket_start)
    if bucket_start == -1 or bucket_end == -1:
        break
    img2tag_mapping[t_i] = disc_time_data[bucket_start:bucket_end + 1]
    
    prev_start = bucket_end + 1
    
pp.pprint(img2tag_mapping)

defaultdict(<class 'list'>,
            {1: [12],
             2: [13],
             3: [14],
             4: [15],
             5: [16],
             6: [17],
             7: [18],
             8: [19],
             9: [20],
             10: [21],
             11: [22],
             12: [23],
             13: [24],
             14: [25],
             15: [26],
             16: [27],
             17: [28],
             18: [29],
             19: [30],
             20: [31],
             21: [32],
             22: [33],
             23: [34],
             24: [35],
             25: [36],
             26: [37],
             27: [38],
             28: [39],
             29: [40],
             30: [41],
             31: [42],
             32: [43],
             33: [44],
             34: [45],
             35: [46],
             36: [47],
             37: [48],
             38: [49],
             39: [50],
             40: [51],
             41: [52],
             42: [53],
             43

In [14]:
with open('run1_meta.txt') as meta:
    img_start = 1 << 64
    img_end = 0
    for line in meta:
        val = float(line.strip())
        img_start = min(img_start, val)
        img_end = max(img_end, val)

In [17]:
print(f'{rfid_start/1000:.2f} to {rfid_end/1000:.2f}')
print(f'{img_start:.2f} to {img_end:.2f}')

1541462369.32 to 1541462385.45
1541462372.26 to 1541462381.23


In [ ]:
E20038EBB5953DC9B7693E9C
E20038EBB5953F49B7693EA2